<a href="https://colab.research.google.com/github/tkhan3/airo/blob/master/hummingbird_cpu2gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hummingbird-ml

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 7.4MB 19.0MB/s 


Create a fake dataset for a classification problem. Our dataset 50 variables with 100000 records in 3 classes.

Randomforest Classifier from Scikit-learn will be trained using this data.

In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
num_classes = 3
X = np.random.rand(100000, 50)
y = np.random.randint(num_classes, size=100000)
randomForestCPU = RandomForestClassifier(n_estimators=10, max_depth=10)
randomForestCPU.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [3]:
%%timeit
randomForestCPU.predict_proba(X)

10 loops, best of 3: 95.7 ms per loop


In [7]:
from hummingbird.ml import convert
# Use Hummingbird to convert the model to PyTorch
randomForestGPU = convert(randomForestCPU, 'pytorch')
randomForestGPU.to('cuda')

In [10]:
%%timeit
randomForestGPU.predict_proba(X)

The slowest run took 8.76 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 12.6 ms per loop
